In [1]:
import time
import pandas as  pd

start_time = time.time()
filepath = 'G:\\내 드라이브\\DataSet\\_최종 병합 파일\\서울특별시 공공자전거 대여소별 이용정보(시간대별)\\'
filename = '시간대별_2021'
df_parquet = pd.read_parquet(filepath + filename + '.parquet')
print("read_parquet time :", time.time() - start_time)

read_parquet time : 4.739560127258301


In [3]:
df_parquet.head()

,대여일자,대여시간,대여소번호,대여소명,이용건수,이동거리(M),이용시간(분)
0,2021-01-01,0,1347,1347. 길음래미안아파트817동 상가 앞,2,992.78,13.0
1,2021-01-01,0,1349,1349. 월계2교 버스정류장 앞,1,1127.50,10.0
2,2021-01-01,0,1168,1168. 마곡엠밸리10단지 앞,1,0.00,7.0
3,2021-01-01,0,2619,2619. 석촌고분역 4번출구,1,1658.34,14.0
4,2021-01-01,0,536,536. 행당역 2번출구 앞,1,743.88,5.0


In [11]:
df_parquet.info() # 이용건수 -> int로 변경하기

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25421803 entries, 0 to 25421802
Data columns (total 7 columns):
 #   Column   Dtype  
---  ------   -----  
 0   대여일자     object 
 1   대여시간     int64  
 2   대여소번호    int64  
 3   대여소명     object 
 4   이용건수     object 
 5   이동거리(M)  float64
 6   이용시간(분)  float64
dtypes: float64(2), int64(2), object(3)
memory usage: 1.3+ GB


In [5]:
df_parquet.count()

대여일자       25421803
대여시간       25421803
대여소번호      25421803
대여소명       25421803
이용건수       25421803
이동거리(M)    25421789
이용시간(분)    25421727
dtype: int64

In [13]:
df_parquet.describe()

,대여시간,대여소번호,이동거리(M),이용시간(분)
count,2.542180e+07,2.542180e+07,2.542179e+07,2.542173e+07
mean,1.445541e+01,1.714753e+03,3.094494e+03,2.837335e+01
std,5.535993e+00,1.266433e+03,4.375321e+03,3.345653e+01
min,0.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00
25%,1.000000e+01,6.770000e+02,7.708800e+02,8.000000e+00
50%,1.600000e+01,1.384000e+03,1.678980e+03,1.600000e+01
75%,1.900000e+01,2.431000e+03,3.672380e+03,3.700000e+01
max,2.300000e+01,9.999900e+04,4.731984e+06,2.701000e+03


In [15]:
pd.options.display.float_format = '{:3}'.format

In [19]:
round(df_parquet.describe(), 1)

,대여시간,대여소번호,이동거리(M),이용시간(분)
count,25421803.0,25421803.0,25421789.0,25421727.0
mean,14.5,1714.8,3094.5,28.4
std,5.5,1266.4,4375.3,33.5
min,0.0,3.0,0.0,0.0
25%,10.0,677.0,770.9,8.0
50%,16.0,1384.0,1679.0,16.0
75%,19.0,2431.0,3672.4,37.0
max,23.0,99999.0,4731983.6,2701.0


In [7]:
df_parquet.isnull().sum()

대여일자        0
대여시간        0
대여소번호       0
대여소명        0
이용건수        0
이동거리(M)    14
이용시간(분)    76
dtype: int64

## 1. 필요한 컬럼확인하기

### 1) 이용건수

In [23]:
df_parquet['이용건수'].value_counts()

이용건수
1         20614051
1.0        2287666
2          1884267
3           302055
2.0         186032
            ...   
48.31            1
31.63            1
13.24            1
168.37           1
29               1
Name: count, Length: 120, dtype: int64

In [25]:
df_parquet[df_parquet['이용건수'] == '\\N']

,대여일자,대여시간,대여소번호,대여소명,이용건수,이동거리(M),이용시간(분)
777566,2021-02-02,18,314,"314. ?뭏냘測鍮抉解?,정기권""",\N,53.0,NaN


### - 이용건수가 null값인 행 삭제하기

In [27]:
df = df_parquet.drop(df_parquet[df_parquet['이용건수'] == '\\N'].index, axis=0)

In [29]:
df.shape[0]

25421802

In [31]:
df.count()

대여일자       25421802
대여시간       25421802
대여소번호      25421802
대여소명       25421802
이용건수       25421802
이동거리(M)    25421788
이용시간(분)    25421727
dtype: int64

In [35]:
df.isnull().sum()

대여일자        0
대여시간        0
대여소번호       0
대여소명        0
이용건수        0
이동거리(M)    14
이용시간(분)    75
dtype: int64

### - '이용시간(분)'이 null값인 행은 데이터가 밀림

In [37]:
df[df['이용시간(분)'].isnull()]

,대여일자,대여시간,대여소번호,대여소명,이용건수,이동거리(M),이용시간(분)
756299,2021-02-01,23,2251,2251. 더케이호텔 입구(양재2),0.00,3.0,NaN
760071,2021-02-02,7,231,231. 남부고용노동지청 남측,11.45,3.0,NaN
761332,2021-02-02,8,1822,1822. 서울 시흥동우체국 앞,128.54,38.0,NaN
771340,2021-02-02,15,324,"324. 신세계백화점 본점 ??,정기권""",31.13,9.0,NaN
828755,2021-02-04,19,3513,3513. 상왕십리역 1번출구,97.63,29.0,NaN
...,...,...,...,...,...,...,...
3581228,2021-03-31,17,519,"519. 양지사?타?,정기권""",34.88,6.0,NaN
3598575,2021-03-31,19,3116,"3116.신촌역 4번출??,정기권""",225.21,42.0,NaN
3606015,2021-03-31,20,792,792.목동트라팰리스 웨스턴에비뉴,62.93,18.0,NaN
3609716,2021-03-31,21,3541,3541. 커먼그라운드,0.19,NaN,NaN


### - '이용기간(분)'이 null값인 행 삭제하기

In [39]:
df = df.dropna(subset=['이용시간(분)'])

In [43]:
df.shape

(25421727, 7)

### - 데이터 타입이 object인 '이용건수' 컬럼을 int로 변경하기

In [49]:
print(df['이용건수'].dtype)

object


In [53]:
df['이용건수'].value_counts()

이용건수
1       20614051
1.0      2287666
2        1884267
3         302055
2.0       186032
4          73285
3.0        27058
5          22308
6           8323
4.0         5919
7           3541
8           1771
5.0         1635
9            984
6.0          639
10           556
11           330
7.0          285
12           197
8.0          155
13           140
9.0           97
14            88
15            59
10.0          51
16            46
17            32
11.0          29
13.0          23
18            19
12.0          17
20            12
14.0           9
15.0           8
19             8
21             7
23             5
22             4
17.0           4
16.0           3
18.0           2
28             2
21.0           1
25             1
19.0           1
24             1
29             1
Name: count, dtype: int64

In [55]:
df['이용건수'] = df['이용건수'].astype(float).astype(int)

In [57]:
df['이용건수'].dtype

dtype('int32')

In [59]:
df.head()

,대여일자,대여시간,대여소번호,대여소명,이용건수,이동거리(M),이용시간(분)
0,2021-01-01,0,1347,1347. 길음래미안아파트817동 상가 앞,2,992.78,13.0
1,2021-01-01,0,1349,1349. 월계2교 버스정류장 앞,1,1127.5,10.0
2,2021-01-01,0,1168,1168. 마곡엠밸리10단지 앞,1,0.0,7.0
3,2021-01-01,0,2619,2619. 석촌고분역 4번출구,1,1658.34,14.0
4,2021-01-01,0,536,536. 행당역 2번출구 앞,1,743.88,5.0


In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25421727 entries, 0 to 25421802
Data columns (total 7 columns):
 #   Column   Dtype  
---  ------   -----  
 0   대여일자     object 
 1   대여시간     int64  
 2   대여소번호    int64  
 3   대여소명     object 
 4   이용건수     int32  
 5   이동거리(M)  float64
 6   이용시간(분)  float64
dtypes: float64(2), int32(1), int64(2), object(2)
memory usage: 1.4+ GB


In [77]:
df['이용건수'].value_counts()

이용건수
1     22901717
2      2070299
3       329113
4        79204
5        23943
6         8962
7         3826
8         1926
9         1081
10         607
11         359
12         214
13         163
14          97
15          67
16          49
17          36
18          21
20          12
19           9
21           8
23           5
22           4
28           2
25           1
24           1
29           1
Name: count, dtype: int64

### 2) 대여일자

In [79]:
df['대여일자'].value_counts()

대여일자
2021-09-02    208866
2021-04-28    182940
2021-04-27    137469
2021-09-24    115696
2021-09-17    114034
               ...  
2021-01-08      7679
2021-03-04      6599
2021-01-07      5350
2021-03-01      2750
2021-05-16      2213
Name: count, Length: 363, dtype: int64

In [83]:
df['대여시간'].isnull().sum()

0

In [85]:
df[df['대여시간'] == '\\N']

,대여일자,대여시간,대여소번호,대여소명,이용건수,이동거리(M),이용시간(분)


### 3) 대여소 번호

In [88]:
df['대여소번호'].value_counts()

대여소번호
502      68279
207      65058
2102     56857
2715     56771
1210     52112
         ...  
1114         2
1476         2
2918         1
9990         1
90002        1
Name: count, Length: 2653, dtype: int64

In [90]:
df['대여소번호'].isnull().sum()

0

In [92]:
df[df['대여소번호'] == '\\N']

,대여일자,대여시간,대여소번호,대여소명,이용건수,이동거리(M),이용시간(분)


### 4) 대여소명

In [95]:
df['대여소명'].value_counts()

대여소명
502. 뚝섬유원지역 1번출구 앞         68278
207. 여의나루역 1번출구 앞          65057
2102. 봉림교 교통섬              56857
2715.마곡나루역 2번 출구           56771
1210. 롯데월드타워(잠실역2번출구 쪽)    52111
                           ...  
4481. 올림픽?貶?북2문 2              1
3672.그라시움 근린공원                 1
424. 롯데하이마??(상암월드컵점)           1
2177. 신대?嚥?2번 출구               1
220. 미성아파트 A??앞                1
Name: count, Length: 2732, dtype: int64

In [97]:
df['대여소번호'].isnull().sum()

0

In [99]:
df[df['대여소번호'] == '\\N']

,대여일자,대여시간,대여소번호,대여소명,이용건수,이동거리(M),이용시간(분)


## 3. 전처리한 데이터셋 parquet으로 저장하기

In [103]:
path_dateset = 'G:/내 드라이브/DataSet/'
path_sub = '_최종 병합 파일/서울특별시 공공자전거 대여소별 이용정보(시간대별)/'

file_path = path_dateset + path_sub

df.to_parquet(file_path + '시간대별_2021.parquet', index=False)

In [106]:
import time
import pandas as  pd

start_time = time.time()
filepath = 'G:\\내 드라이브\\DataSet\\_최종 병합 파일\\서울특별시 공공자전거 대여소별 이용정보(시간대별)\\'
filename = '시간대별_2021'
df_parquet = pd.read_parquet(filepath + filename + '.parquet')
print("read_parquet time :", time.time() - start_time)

read_parquet time : 3.670543909072876


In [108]:
df_parquet.head()

,대여일자,대여시간,대여소번호,대여소명,이용건수,이동거리(M),이용시간(분)
0,2021-01-01,0,1347,1347. 길음래미안아파트817동 상가 앞,2,992.78,13.0
1,2021-01-01,0,1349,1349. 월계2교 버스정류장 앞,1,1127.5,10.0
2,2021-01-01,0,1168,1168. 마곡엠밸리10단지 앞,1,0.0,7.0
3,2021-01-01,0,2619,2619. 석촌고분역 4번출구,1,1658.34,14.0
4,2021-01-01,0,536,536. 행당역 2번출구 앞,1,743.88,5.0


In [110]:
df_parquet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25421727 entries, 0 to 25421726
Data columns (total 7 columns):
 #   Column   Dtype  
---  ------   -----  
 0   대여일자     object 
 1   대여시간     int64  
 2   대여소번호    int64  
 3   대여소명     object 
 4   이용건수     int32  
 5   이동거리(M)  float64
 6   이용시간(분)  float64
dtypes: float64(2), int32(1), int64(2), object(2)
memory usage: 1.2+ GB


In [112]:
df_parquet.isnull().sum()

대여일자       0
대여시간       0
대여소번호      0
대여소명       0
이용건수       0
이동거리(M)    0
이용시간(분)    0
dtype: int64